In [27]:
import sys 
import os
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.features import preprocessing
from src.features import sequences
from src.features import knowledge

from pathlib import Path

In [28]:
data_path = Path('/home/ralmasri/projects/Thesis/Domain-Guided-Monitoring/data/')
csv_path = data_path / "logs_aggregated_concurrent.csv"
size = 50
# data_df = pd.read_csv(csv_path).head(size) if size > 0  else pd.read_csv(csv_path)
subset_path = data_path / f"{size}_logs_aggregated_concurrent.csv" if size > 0 else csv_path

In [29]:
huawei_config = preprocessing.HuaweiPreprocessorConfig()
huawei_config.aggregated_log_file = subset_path
sequence_preprocessor = preprocessing.ConcurrentAggregatedLogsPreprocessor(huawei_config)
sequence_column_name = sequence_preprocessor.sequence_column_name
sequence_df = sequence_preprocessor.load_data()
sequence_df

Generating DRAIN clusters from log_df: 100%|██████████| 15/15 [00:00<00:00, 1687.12it/s]


,num_logs,num_events,all_events,attributes,coarse_log_cluster_template,fine_log_cluster_template
0,50,50,[[coarse_log_cluster_template#19 nov 2019 17 *...,[[url_cluster_template#resource providers 45ba...,[[coarse_log_cluster_template#19 nov 2019 17 *...,[[fine_log_cluster_template#19 nov 2019 17 * *...


In [30]:
sequence_config = sequences.SequenceConfig()
transformer = sequences.NextSequenceTransformer(sequence_config)
metadata = transformer.collect_metadata(sequence_df=sequence_df, sequence_column_name='all_events')
metadata

In [35]:
causality_preprocessor = preprocessing.ConcurrentAggregatedLogsCausalityPreprocessor(
    config=huawei_config,
)
causality_df = causality_preprocessor.load_data()
print(causality_df.shape)
causality_df.head(3)

Generating causality df from counted causality: 100%|██████████| 66/66 [00:00<00:00, 14936.82it/s]


(793, 4)


,parent_id,parent_name,child_id,child_name
0,coarse_log_cluster_template#get v20 networks s...,get v20 networks shared true http 11 status 20...,coarse_log_cluster_template#get v20 ports fiel...,get v20 ports fields binding 3ahost id fields ...
1,coarse_log_cluster_template#get v20 networks s...,get v20 networks shared true http 11 status 20...,Hostname#wally113,wally113
2,coarse_log_cluster_template#get v20 networks s...,get v20 networks shared true http 11 status 20...,python_module#neutron.wsgi,neutron.wsgi


In [32]:
knowledge_config = knowledge.KnowledgeConfig()
knowledge_config.file_knowledge = data_path / "file_knowledge.json"
causality_knowledge = knowledge.CausalityKnowledge(config=knowledge.KnowledgeConfig())
extended_vocab = causality_knowledge._build_extended_vocab(causality_df, metadata.x_vocab)
len(causality_knowledge.extended_vocab)

67

In [33]:
node = causality_knowledge.nodes[2]
print(node)

Node for idx 2 (label: coarse_log_cluster_template#19 nov 2019 17 03 * 0100 * * * * * * * * * * * * *, name: 19 nov 2019 17 03 * 0100 * * * * * * * * * * * * *)
<-Parent nodes: 
->Child nodes: 
